# Bomb Calorimitery Virtual Experiment

In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
import numpy as np
from IPython.display import display, Markdown, FileLink, clear_output

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

In [ ]:
with open(".lab.json") as infile:
    jsdata = json.load(infile)

params = jsdata["cal1"]

In [ ]:
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

params["roomT"] = random.gauss(298,2)
params["slope"] = (2*random.random()) / params["nTime"]

# sample properties
params["mass"] = 0.0
params["mM"] = 0. # mole/g
params["n1"] = 0 
params["n2"] = 0 
params["dn"] = 0
params["dH"] = 0

In [ ]:
w = ipw.Dropdown(
    options=['None',
             'Benzoic acid', 
             'Sucrose', 
             'Naphthalene'],
    value='None',
    description='Sample:'
)

def sample_select(change):
    if change['new'] == "Benzoic acid":
        params["mM"] = 122.123
        params["n1"] = 7 
        params["n2"] = 2 
        params["dn"] = 0
        params["dH"] = -386000
        
    if change['new'] == "Sucrose":
        params["mM"] = 342.3
        params["n1"] = 12 
        params["n2"] = 11 
        params["dn"] = 1
        params["dH"] = -2221200

    if change['new'] == "Naphthalene":
        params["mM"] = 128.17
        params["n1"] = 10 
        params["n2"] = 2 
        params["dn"] = 2
        params["dH"] = -77000 

w.observe(sample_select)

In [ ]:
def run_experiment():
    
    # Error handing
    if float(params["mM"]) < 0.1:
        with out_Error:
            print("Please select a sample to use in the experiment")
        raise StopExecution

    if params["mass"] < 0.1:
        with out_Error:
            print("Please prepare a tablet first")
        raise StopExecution

    moles = params["mass"] / 1000 / params["mM"]

    # combustion enthalpy
    # nH{co2} + mH{h2o} - H = DcH
    DcH = params["n1"]*params["dH_co2"] + params["n2"]*params["dH_h2o"] - params["dH"]
    
    dH = DcH * moles
    dnrt = moles * params["R"] * params["roomT"] * params["dn"] 
    dU = dH - dnrt
    deltaT = -dU / params["cc"] #+ params["roomT"] 
   
    res = pd.DataFrame(columns=["Time [min]" , "T [K]"])
    for i in range(0, params["nTime"]):
        var_list = []
        var_list.append(i/10)
        tempOut = params["roomT"] + random.gauss(0,params["error"]) + params["slope"] * i
        if (i > params["mixTime"]):
            tempOut = tempOut + deltaT * (1-math.exp(-(i-params["mixTime"])/params["relaxTime"] ))
            
        var_list.append(tempOut)
        res.loc[len(res)] = var_list
    
    res.to_csv(respath.value, index=False)
    with out_P:
        display(res.tail(params["nTime"]))


In [ ]:
out_Error = ipw.Output()
out_P = ipw.Output()
out_T = ipw.Output()

fileName = "results.csv"
respath = ipw.Text(fileName, layout=ipw.Layout(width="150px"))
f = ipw.HBox([ipw.Label('Output filename : '),respath])

def reset(btn):
    if os.path.exists(respath.value):
        os.remove(respath.value)
    
    with out_Error:
        out_Error.clear_output()
        
    with out_P:
        out_P.clear_output()
        
    with out_T:
        out_T.clear_output()
    
    clear_output()
    create_ipw()

def calc(btn):
    out_P.clear_output()
    run_experiment()
    local_file = FileLink(respath.value, result_html_prefix="Click here to download: ")
    with out_P:
        display(local_file)

def make_tablet(btn):
    out_T.clear_output()

    params["mass"] = random.gauss(1000,100)
    
    with out_T:
        print("Mass of the tablet (mg) :",params["mass"])
        
# interactive buttons ---
btn_calc = ipw.Button(description="Perform Experiment", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

btn_reset = ipw.Button(description="Reset Experiment", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

btn_tab = ipw.Button(description="Make tablet", layout=ipw.Layout(width="150px"))
btn_tab.on_click(make_tablet)

# ---
# reset(btn_reset)


# -- output widgets
def create_ipw():
    rows = []
    rows.append(ipw.HBox([f]))
    rows.append(ipw.HBox([w]))
    rows.append(ipw.HBox([btn_tab]))
    rows.append(ipw.HBox([out_T]))
    rows.append(ipw.HBox([btn_reset,btn_calc]))
    rows.append(ipw.HBox([out_Error]))
    rows.append(ipw.HBox([out_P]))
    display(ipw.VBox(rows))

create_ipw()
